In [ ]:
Datadirectory = "training/"
Classes = ["0","1","2","3","4","5","6"]

import random

training_Data = []
img_size = 224
desired_per_category = 1000 
total_desired_size = 6000 

def create_training_Data():
    samples_per_category = {category: 0 for category in Classes}

    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category)
        
        for img in os.listdir(path):
            try:
                if samples_per_category[category] < desired_per_category:
                    img_array = cv2.imread(os.path.join(path, img))
                    new_array = cv2.resize(img_array, (img_size, img_size))
                    training_Data.append([new_array, class_num])
                    samples_per_category[category] += 1
            except Exception as e:
                pass

            if sum(samples_per_category.values()) >= total_desired_size:
                # Stop adding samples once the total desired size is reached
                break

        if sum(samples_per_category.values()) >= total_desired_size:
            # Stop iterating through categories once the total desired size is reached
            break

create_training_Data()
import random

random.shuffle(training_Data)

x = []
y= []
for features,label in training_Data:
    x.append(features)
    y.append(label)
x = np.array(x).reshape(-1 , img_size , img_size , 3) #converting it to 4 dimension

x=x/255.0

Y = np.array(y)

import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers

model = tf.keras.applications.MobileNetV2() ## pre-trained model

base_input = model.layers[0].input

base_output = model.layers[-2].output

final_output = layers.Dense(128)(base_output) ## adding new layer , after the output of global pooling layer
final_output = layers.Activation('relu')(final_output) ## activation function
final_output = layers.Dense(64)(final_output) 
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7,activation="softmax")(final_output) ## classes are 7 / classification layer

new_model = keras.Model(inputs = base_input , outputs = final_output)

new_model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

new_model.fit(x, Y, epochs=25)

new_model.save('my_mmodel_64p35.h5')


In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

# Load the pre-trained model
new_model = tf.keras.models.load_model('my_mmodel_64p35.h5')

# Load the face cascade classifier
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Set up the webcam
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Initialize the face_roi variable
face_roi = None

# Main loop
while True:
    try:
        # Read a frame from the webcam
        ret, frame = cap.read()

        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = faceCascade.detectMultiScale(gray, 1.1, 4)

        for x, y, w, h in faces:
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = frame[y:y + h, x:x + w]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            facess = faceCascade.detectMultiScale(roi_gray)

            if len(facess) == 0:
                print("Face not detected")
                break
            else:
                for (ex, ey, ew, eh) in facess:
                    face_roi = roi_color[ey:ey + eh, ex:ex + ew]

        # Check if face_roi is not None before processing
        if face_roi is not None:
            # Preprocess the face image for the model
            final_image = cv2.resize(face_roi, (224, 224))
            final_image = np.expand_dims(final_image, axis=0)
            final_image = final_image / 255.0

            # Make predictions using the model
            Predictions = new_model.predict(final_image)

            # Determine the emotion status
            status = ""
            emotions = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
            status = emotions[np.argmax(Predictions)]

            # Display the emotion status on the frame
            x1, y1, w1, h1 = 0, 0, 175, 175
            cv2.rectangle(frame, (x1, x1), (x1 + w1, y1 + h1), (0, 0, 0), -1)
            cv2.putText(frame, status, (x1 + int(w1 / 10), y1 + int(h1 / 2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255))

        # Display the frame
        cv2.imshow('Face Emotion Recognition', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(2) & 0xFF == ord('q'):
            break

    except Exception as e:
        # Handle any exceptions that occur and print an error message
        print(f"An error occurred: {e}")
        cap.release()
        cv2.destroyAllWindows()

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()
